In [ ]:
import urllib.parse
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

params=urllib.parse.quote_plus('Driver={ODBC Driver 17 for SQL Server};Server=DESKTOP-4JUNPCV;Database=News;Trusted_Connection=yes;')

app=Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI']=f'mssql+pyodbc:///?odbc_connect={params}'
db=SQLAlchemy(app)

class User(db.Model):
    __tablename__='stories'
    id=db.Column(db.INTEGER,primary_key=True)
    category = db.Column(db.String('max'),unique=False,nullable=False)
    subcategory = db.Column(db.String('max'),unique=False,nullable=False)
    title=db.Column(db.String('max'),unique=False,nullable=False)
    date=db.Column(db.String('max'),unique=False,nullable=False)
    abstract=db.Column(db.String('max'),unique=False,nullable=False)
    contents=db.Column(db.String('max'),unique=False,nullable=False)
    url=db.Column(db.String('max'),unique=False,nullable=False)
    

    
    def __init__(self,category,subcategory,title,date,abstract,contents,url):
        self.category=category
        self.subcategory=subcategory
        self.title=title
        self.date=date
        self.abstract=abstract
        self.contents=contents
        self.url=url

class User(db.Model):
    __tablename__='videos'
    id=db.Column(db.INTEGER,primary_key=True)
    title=db.Column(db.String('max'),unique=False,nullable=False)
    date=db.Column(db.String('max'),unique=False,nullable=False)
    abstract=db.Column(db.String('max'),unique=False,nullable=False)
    contents=db.Column(db.String('max'),unique=False,nullable=False)
    url=db.Column(db.String('max'),unique=False,nullable=False)
    
    
    def __init__(self,title,date,abstract,contents,url):
        self.title=title
        self.date=date
        self.abstract=abstract
        self.contents=contents
        self.url=url

if __name__=='__main__':
    with app.app_context():
        db.drop_all()
        db.create_all()      

In [ ]:
#文章類
import pyodbc
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time



conn=pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};''Server=DESKTOP-4JUNPCV;''Database=News;''Trusted_Connection=yes;')
chrome_path = 'C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver.exe'  
service = Service(chrome_path)
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(10)

url = 'https://edition.cnn.com/'
driver.get(url)
cur = conn.cursor()
cur.fast_executemany = False
time.sleep(2)

driver.find_element(By.ID, 'headerSearchIcon').click() 

keyword = 'Artificial Intelligence'
driver.find_element(By.CLASS_NAME, 'search-bar__input').send_keys(keyword)
driver.find_element(By.CLASS_NAME, 'search-bar__submit').click() 
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="search"]/div[1]/div[2]/div/div/ul/li[2]/label').click() 
time.sleep(10)

#第一頁
for x in range(1,10):                 
    titles = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[1]/span').text  #標題
    publication_dates =  driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[2]').text   #日期  
    abstracts = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[3]').text    #文章摘要 
    elements = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]')
    urls = elements.get_attribute('href')   #連結
    
    driver.get(urls)    
    try:
        categories = driver.find_element(By.XPATH, '/html/body/div[3]/section[3]/div[1]/a[1]').text  #類別
        subcategories = driver.find_element(By.XPATH, '/html/body/div[3]/section[3]/div[1]/a[2]').text  #子類別
    except:
        categories = " "
        subcategories = " "

    try:
            a = driver.find_element(By.XPATH, '/html/body/div[3]/section[4]/section[1]/section[1]/article/section/main/div[2]/div[1]')
            time.sleep(2)
            paragraphs = a.find_elements(By.TAG_NAME, 'p')
            time.sleep(2)
            content_str = [p.text for p in paragraphs]
            content = "\n".join(content_str)
    except:
        content = " "

    cur.execute('''INSERT INTO stories(category,subcategory,title,date,abstract,contents,url) VALUES(?,?,?,?,?,?,?);''',(categories,subcategories,titles,publication_dates,abstracts,content,urls))
    conn.commit()
    time.sleep(1)

    driver.back()
    time.sleep(1)

for y in range(1,7):               
    pages=driver.find_element(By.XPATH,'//*[@id="search"]/div[2]/div/div[4]/div/div[3]').click()
    time.sleep(10) 
    for z in range(1,10):                     
        titles = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[1]/span').text  #標題
        publication_dates =  driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[2]').text   #日期  
        abstracts = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[3]').text    #文章摘要 
        elements = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]')
        urls = elements.get_attribute('href')   #連結
        
        driver.get(urls)
        try:
            categories = driver.find_element(By.XPATH, '/html/body/div[3]/section[3]/div[1]/a[1]').text  #類別
            subcategories = driver.find_element(By.XPATH, '/html/body/div[3]/section[3]/div[1]/a[2]').text  #子類別
        except:
            categories = " "
            subcategories = " "

        try:
            a = driver.find_element(By.XPATH, '/html/body/div[3]/section[4]/section[1]/section[1]/article/section/main/div[2]/div[1]')
            time.sleep(2)
            paragraphs = a.find_elements(By.TAG_NAME, 'p')
            time.sleep(2)
            content_str = [p.text for p in paragraphs]
            content = "\n".join(content_str)
        except:
            content = " "

        cur.execute('''INSERT INTO stories(category,subcategory,title,date,abstract,contents,url) VALUES(?,?,?,?,?,?,?);''',(categories,subcategories,titles,publication_dates,abstracts,content,urls))
        conn.commit()
        time.sleep(1)

        driver.back()
        time.sleep(1)

driver.quit()

In [ ]:
#影片類
import pyodbc
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

conn=pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};''Server=DESKTOP-4JUNPCV;''Database=News;''Trusted_Connection=yes;')
chrome_path = 'C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver.exe'  
service = Service(chrome_path)
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(10)

url = 'https://edition.cnn.com/'
driver.get(url)
cur = conn.cursor()
time.sleep(2)

driver.find_element(By.ID, 'headerSearchIcon').click() 

keyword = 'Artificial Intelligence'
driver.find_element(By.CLASS_NAME, 'search-bar__input').send_keys(keyword)
driver.find_element(By.CLASS_NAME, 'search-bar__submit').click() 
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="search"]/div[1]/div[2]/div/div/ul/li[3]/label').click() 
time.sleep(10)

#第一頁
for x in range(1,7):                 
    titles = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[1]/span').text  #標題
    publication_dates =  driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[2]').text   #日期  
    abstracts = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[3]').text    #文章摘要 
    elements = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]')
    urls = elements.get_attribute('href')   #連結
    
    driver.get(urls)           
    a = driver.find_elements(By.XPATH, '/html/body/div[1]/section[4]/section/div/section/div/div/div[1]/div/div[3]/div[5]')
    time.sleep(2)
    for item in a:
        content = item.text   #內文
    cur.execute('''INSERT INTO videos(title,date,abstract,contents,url) VALUES(?,?,?,?,?);''',(titles,publication_dates,abstracts,content,urls))
    conn.commit()
    time.sleep(1)

    driver.back()
    time.sleep(1)
    
for y in range(1,9):               
    pages=driver.find_element(By.XPATH,'//*[@id="search"]/div[2]/div/div[4]/div/div[3]').click()
    time.sleep(10) 
    for z in range(1,11):                     
        titles = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[1]/span').text  #標題
        publication_dates =  driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[2]').text   #日期  
        abstracts = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[3]').text    #文章摘要 
        elements = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]')
        urls = elements.get_attribute('href')   #連結

        driver.get(urls)           
        a = driver.find_elements(By.XPATH, '/html/body/div[1]/section[4]/section/div/section/div/div/div[1]/div/div[3]/div[5]')
        time.sleep(2)
        for item in a:
            content = item.text   #內文  
        cur.execute('''INSERT INTO videos(title,date,abstract,contents,url) VALUES(?,?,?,?,?);''',(titles,publication_dates,abstracts,content,urls))
        conn.commit()
        time.sleep(1)

        driver.back()
        time.sleep(1)

driver.quit()